## Obtain text format ANOVA results
TODOs (not implemented yet):
- between subject factors
- readout group factor from between subject tables
- readout the main effect directions
- clean up: move the functions to offline script
- implement multiple groups

0. adjust if necessary

In [1]:
alpha_level = 0.05
debug = True

1. export all SPSS ANOVA result tables "Tests der Innersubjekteffekte" as .txt files  
(right-click on table and select "Exportieren...")
![title](instruction.jpg)  

2. Select the file by running the cell below:

In [2]:
import numpy as np
import pandas as pd

In [3]:
import tkinter as tk
from tkinter import filedialog
from pathlib import Path

if debug:
    filename = "RT_ANOVA_SPSS_all.htm"
else:
    # hiding the root tkinter window (not needed)
    root = tk.Tk()
    root.withdraw()
    # open the file selector
    filename = filedialog.askopenfilename(initialdir = ".", #"R:/AG-Beste-Studien/",
                                          title = "Select file",
                                          filetypes = (('Text files','*.htm'),("all files","*.*")))
filename

'RT_ANOVA_SPSS_all.htm'

3. collect all the tables from exported HTML

In [4]:
# other approach: read html tables to pandas

from bs4 import BeautifulSoup
f = open(filename, "r", encoding="utf8")
html = f.read()
soup = BeautifulSoup(html, "lxml")
captions = soup.find_all("caption")
captions = [str(c).split(">")[1].split("<")[0] for c in captions]
print(captions)

['Hinweise', 'Innersubjektfaktoren', 'Zwischensubjektfaktoren', 'Deskriptive Statistiken', 'Multivariate Tests', 'Mauchly-Test auf Sphärizität', 'Tests der Innersubjekteffekte', 'Tests der Innersubjektkontraste', 'Tests der Zwischensubjekteffekte', '1. Gesamtmittelwert', 'Schätzungen', 'Paarweise Vergleiche', 'Tests auf Univariate', 'Schätzungen', 'Paarweise Vergleiche', 'Multivariate Tests', 'Schätzungen', 'Paarweise Vergleiche', 'Multivariate Tests', '5. group * comp_0_3', '6. group * rep_alt', '7. comp_0_3 * rep_alt', '8. group * comp_0_3 * rep_alt']


In [31]:
tables = {}
for c in captions:
    tables[c] = pd.read_html(filename, decimal=",", thousands=".", match=c.replace("*", "\*"))[0]

4. collect variables from tables

In [32]:
within_f = []
if "Innersubjektfaktoren" in captions:
    df = tables["Innersubjektfaktoren"]
    within_f = [ind[1] for ind in df.columns[:-1]]
within_f

['comp_0_3', 'rep_alt']

In [33]:
between_f = []
if "Zwischensubjektfaktoren" in captions:
    df = tables["Zwischensubjektfaktoren"]
    # TODO implement multiple groups
    between_f.append(df.iloc[0,0])
    
between_f

['group']

3. load in the data by running the cell below

In [34]:
captions

['Hinweise',
 'Innersubjektfaktoren',
 'Zwischensubjektfaktoren',
 'Deskriptive Statistiken',
 'Multivariate Tests',
 'Mauchly-Test auf Sphärizität',
 'Tests der Innersubjekteffekte',
 'Tests der Innersubjektkontraste',
 'Tests der Zwischensubjekteffekte',
 '1. Gesamtmittelwert',
 'Schätzungen',
 'Paarweise Vergleiche',
 'Tests auf Univariate',
 'Schätzungen',
 'Paarweise Vergleiche',
 'Multivariate Tests',
 'Schätzungen',
 'Paarweise Vergleiche',
 'Multivariate Tests',
 '5. group * comp_0_3',
 '6. group * rep_alt',
 '7. comp_0_3 * rep_alt',
 '8. group * comp_0_3 * rep_alt']

In [35]:
ANOVA_df_within = tables["Tests der Innersubjekteffekte"]

4. name the factors as appearing in the article, by running the cell below:

In [36]:
within_factors_text = {}
for fac in within_f:
    within_factors_text[fac] = input(f"Enter a name for the within factor '{fac}':")
    
between_factors_text = {}
for fac in between_f:
    between_factors_text[fac] = input(f"Enter a name for the between factor '{fac}':")

Enter a name for the within factor 'comp_0_3':a
Enter a name for the within factor 'rep_alt':b
Enter a name for the between factor 'group':c


In [57]:
all_factors_text = dict(within_factors_text, **between_factors_text)

5. run the cell below and copy the output as necessary in your manuscript

In [37]:
ANOVA_df_within.head(10)

Maß:MASS_1                                                        \
             Quelle                Quelle.1 Typ III Quadratsumme       df   
0          comp_0_3  Sphärizität angenommen             4171.946        1   
1          comp_0_3      Greenhouse-Geisser             4171.946    1.000   
2          comp_0_3        Huynh-Feldt (HF)             4171.946    1.000   
3          comp_0_3             Untergrenze             4171.946    1.000   
4  comp_0_3 * group  Sphärizität angenommen              172.088        1   
5  comp_0_3 * group      Greenhouse-Geisser              172.088    1.000   
6  comp_0_3 * group        Huynh-Feldt (HF)              172.088    1.000   
7  comp_0_3 * group             Untergrenze              172.088    1.000   
8  Fehler(comp_0_3)  Sphärizität angenommen           473782.086      146   
9  Fehler(comp_0_3)      Greenhouse-Geisser           473782.086  146.000   

                                                                              \
  Mittel der Quadrate      F  Sig. Partielles Eta-Quadrat Dezentr. Parameter   
0            4171.946  1.286  .259                   .009              1.286   
1            4171.946  1.286  .259                   .009              1.286   
2            4171.946  1.286  .259                   .009              1.286   
3            4171.946  1.286  .259                   .009              1.286   
4             172.088   .053  .818                   .000               .053   
5             172.088   .053  .818                   .000               .053   
6             172.088   .053  .818                   .000               .053   
7             172.088   .053  .818                   .000               .053   
8            3245.083    NaN   NaN                    NaN                NaN   
9            3245.083    NaN   NaN                    NaN                NaN   

                             
  Beobachtete Trennschärfea  
0                      .203  
1                      .203  
2                      .203  
3                      .203  
4                      .056  
5                      .056  
6                      .056  
7                      .056  
8                       NaN  
9                       NaN

In [38]:
ANOVA_df_within["Maß:MASS_1"]["Sig."]

0                                              .259
1                                              .259
2                                              .259
3                                              .259
4                                              .818
5                                              .818
6                                              .818
7                                              .818
8                                               NaN
9                                               NaN
10                                              NaN
11                                              NaN
12                                             .003
13                                             .003
14                                             .003
15                                             .003
16                                             .808
17                                             .808
18                                             .808
19          

In [42]:
def setup_GG_df(ANOVA_df_w):
    # display factor names for all rows
    """
    sources = [s.rstrip() for s in ANOVA_df["Unnamed: 0"].tolist()]
    for i, source in enumerate(sources[1:]):
        sources[i+1] = sources[i] if len(sources[i+1]) == 0 else sources[i+1]
    ANOVA_df["Unnamed: 0"] = sources
    """
    
    # TODO figure out why this df is not a local copy but affects the reference
    try:
        ANOVA_df_w.columns = ANOVA_df_w.columns.droplevel()
    except ValueError:
        pass

    # limit table to Greenhouse-Geisser correction
    #ANOVA_df["Quelle"] = [e.rstrip() for e in ANOVA_df["Quelle"].tolist()]
    ANOVA_GG_df = ANOVA_df_w[ANOVA_df_w["Quelle.1"] == "Greenhouse-Geisser"]

    # split the error table
    ANOVA_GG_effects_df = ANOVA_GG_df[~ANOVA_GG_df["Quelle"].str.contains("Fehler")]
    ANOVA_GG_errors_df = ANOVA_GG_df[ANOVA_GG_df["Quelle"].str.contains("Fehler")]
    #ANOVA_GG_errors_df.set_index("Unnamed: 0", inplace=True)

    
    ANOVA_GG_effects_df["Sig."] = [float(v) for v in ANOVA_GG_effects_df["Sig."].tolist()]
    ANOVA_GG_effects_df["Sig."] = pd.to_numeric(ANOVA_GG_effects_df["Sig."])
    
    ANOVA_GG_sig_df = ANOVA_GG_effects_df[ANOVA_GG_effects_df["Sig."] <= alpha_level]
    ANOVA_GG_sig_df.index = list(range(len(ANOVA_GG_sig_df)))
    
    ANOVA_GG_nonsig_df = ANOVA_GG_effects_df[ANOVA_GG_effects_df["Sig."] > alpha_level]
    # sort non-significant table to p values
    ANOVA_GG_nonsig_df.sort_values("Sig.", inplace=True)
    ANOVA_GG_nonsig_df.index = list(range(len(ANOVA_GG_nonsig_df)))

    return [ANOVA_GG_sig_df, ANOVA_GG_nonsig_df, ANOVA_GG_errors_df]

In [43]:
tables["Tests der Zwischensubjekteffekte"]

Maß:MASS_1  \
                         Transformierte Variable   
                                          Mittel   
                                          Quelle   
0                                Konstanter Term   
1                                          group   
2                                         Fehler   
3  a. Unter Verwendung von Alpha = ,05 berechnet   

                                                  \
                                                   
                                                   
                            Typ III Quadratsumme   
0                                  193241300.601   
1                                      37038.424   
2                                   10404986.469   
3  a. Unter Verwendung von Alpha = ,05 berechnet   

                                                  \
                                                   
                                                   
                                              df   
0                                              1   
1                                              1   
2                                            146   
3  a. Unter Verwendung von Alpha = ,05 berechnet   

                                                  \
                                                   
                                                   
                             Mittel der Quadrate   
0                                  193241300.601   
1                                      37038.424   
2                                      71267.031   
3  a. Unter Verwendung von Alpha = ,05 berechnet   

                                                  \
                                                   
                                                   
                                               F   
0                                       2711.510   
1                                           .520   
2                                            NaN   
3  a. Unter Verwendung von Alpha = ,05 berechnet   

                                                  \
                                                   
                                                   
                                            Sig.   
0                                           .000   
1                                           .472   
2                                            NaN   
3  a. Unter Verwendung von Alpha = ,05 berechnet   

                                                  \
                                                   
                                                   
                          Partielles Eta-Quadrat   
0                                           .949   
1                                           .004   
2                                            NaN   
3  a. Unter Verwendung von Alpha = ,05 berechnet   

                                                  \
                                                   
                                                   
                              Dezentr. Parameter   
0                                       2711.510   
1                                           .520   
2                                            NaN   
3  a. Unter Verwendung von Alpha = ,05 berechnet   

                                                  
                                                  
                                                  
                       Beobachtete Trennschärfea  
0                                          1.000  
1                                           .111  
2                                            NaN  
3  a. Unter Verwendung von Alpha = ,05 berechnet

In [44]:
all_GG = setup_GG_df(ANOVA_df_within)

C:\Users\a\miniconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\a\miniconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\a\miniconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

In [47]:
all_GG[1]

,Quelle,Quelle.1,Typ III Quadratsumme,df,Mittel der Quadrate,F,Sig.,Partielles Eta-Quadrat,Dezentr. Parameter,Beobachtete Trennschärfea
0,comp_0_3,Greenhouse-Geisser,4171.946,1.000,4171.946,1.286,0.259,.009,1.286,.203
1,rep_alt * group,Greenhouse-Geisser,178.567,1.000,178.567,.060,0.808,.000,.060,.057
2,comp_0_3 * group,Greenhouse-Geisser,172.088,1.000,172.088,.053,0.818,.000,.053,.056
3,comp_0_3 * rep_alt * group,Greenhouse-Geisser,72.481,1.000,72.481,.046,0.831,.000,.046,.055


In [45]:
all_GG[2]

,Quelle,Quelle.1,Typ III Quadratsumme,df,Mittel der Quadrate,F,Sig.,Partielles Eta-Quadrat,Dezentr. Parameter,Beobachtete Trennschärfea
9,Fehler(comp_0_3),Greenhouse-Geisser,473782.086,146.000,3245.083,NaN,NaN,NaN,NaN,NaN
21,Fehler(rep_alt),Greenhouse-Geisser,437854.148,146.000,2999.001,NaN,NaN,NaN,NaN,NaN
33,Fehler(comp_0_3*rep_alt),Greenhouse-Geisser,230760.844,146.000,1580.554,NaN,NaN,NaN,NaN,NaN


In [53]:
# setup the html writer function
from IPython.display import HTML
def write_html_line(effect_name, df1, df2, F_val, p_val, eta_val):
    # prepare standard strings
    all_other = ""
    eta_part = "; &#951;<sub>p</sub><sup>2</sup> = " + eta_val
    p_sign = "="
    F_sign = "="
    # special cases
    if p_val < 0.001:
        p_sign = "&#60;"
        p_val = ".001"
    elif p_val > alpha_level:
        # non significant:
        p_sign = "&#8805;"
        F_sign = "&#8804;"
        all_other = "All other "
        eta_part = ""
    # formatting
    df1 = '%g'%(df1)
    df2 = '%g'%(df2)
    html_text = """
    <b>{effect_name}</b> <br>
    {all_other}F({df1}, {df2}) {F_sign} {F_val}; p {p_sign} {p_val}{eta_part} <br>
    """
    # include the variables
    return html_text.format(effect_name=effect_name, 
                            all_other=all_other, 
                            df1=df1, 
                            df2=df2, 
                            F_sign=F_sign, 
                            F_val=F_val, 
                            p_sign=p_sign, 
                            p_val=p_val, 
                            eta_part=eta_part)

In [65]:
def get_error_index(effect_str):
    # replace spaces
    error_ind = effect_str.replace(" * ", "*")
    # remove between sbj factors
    for bf in between_f:
        error_ind.replace(f"*{bf}", "")
    return f"Fehler({error_ind})"

In [82]:
def print_sign_results(ANOVA_GG_sig_df, e_df):
    display(HTML('<b>SIGNIFICANT RESULTS</b>'))

    for i in range(len(ANOVA_GG_sig_df)):
        factor_name = " &#215 ".join([all_factors_text[p] for p in ANOVA_GG_sig_df["Quelle"][i].split(" * ")])
        
        html_sig = write_html_line(
            factor_name, 
            float(ANOVA_GG_sig_df["df"][i]), 
            float(e_df[e_df["Quelle"] == get_error_index(ANOVA_GG_sig_df["Quelle"][0])]["df"].tolist()[0]),
            float(ANOVA_GG_sig_df["F"][i]), 
            ANOVA_GG_sig_df["Sig."][i], 
            ANOVA_GG_sig_df["Partielles Eta-Quadrat"][i]
           )
        display(HTML(html_sig))

In [83]:
print_sign_results(all_GG[0], all_GG[2])

In [22]:
# gather the important infos

def print_sign_results(ANOVA_GG_sig_df):
    display(HTML('<b>SIGNIFICANT RESULTS</b>'))

    for i in range(len(ANOVA_GG_sig_df)):
        factor_name = " &#215 ".join([factors_text[p] for p in ANOVA_GG_sig_df["Unnamed: 0"][i].split(" * ")])
        html_sig = write_html_line(factor_name, 
            float(ANOVA_GG_sig_df["df"][i]), 
            float(ANOVA_GG_errors_df["df"]["Fehler(" + "*".join(ANOVA_GG_sig_df["Unnamed: 0"][i].split(" * ")) + ")"]),
            float(ANOVA_GG_sig_df["F"][i]), 
            ANOVA_GG_sig_df["Sig."][i], 
            ANOVA_GG_sig_df["Partielles Eta-Quadrat"][i]
           )
        display(HTML(html_sig))

def print_nonsign_results(ANOVA_GG_sig_df):
    display(HTML('<b>NON-SIGNIFICANT RESULTS</b>'))

    factor_name = " &#215 ".join([factors_text[p] for p in ANOVA_GG_nonsig_df["Unnamed: 0"][0].split(" * ")])
    html_nonsig = write_html_line(factor_name, 
        float(ANOVA_GG_nonsig_df["df"][0]), 
        float(ANOVA_GG_errors_df["df"]["Fehler(" + "*".join(ANOVA_GG_nonsig_df["Unnamed: 0"][0].split(" * ")) + ")"]),
        float(ANOVA_GG_nonsig_df["F"][0]), 
        ANOVA_GG_nonsig_df["Sig."][0], 
        ANOVA_GG_nonsig_df["Partielles Eta-Quadrat"][0]
       )
    display(HTML(html_nonsig))

In [19]:
ANOVA_GG_errors_df["df"]["Fehler(" + "*".join(ANOVA_GG_sig_df["Unnamed: 0"][i].split(" * ")) + ")"]

KeyError: 'Fehler(comp_0_3*rep_alt*group)'